# Process the harvested data

Using the Trove Newspaper Harvester, I've downloaded the metadata from 19 million newspaper articles that were published on page one. The Harvester has saved all the metadata into a big (14.6gb) newline-delimited JSON file. In this notebook we'll process the JSON, extracting the fields we want into a CSV file. Then we'll use DuckDB to save the CSV as a (much smaller) parquet file, and create another parquet file that sums the word counts per page and category.

In [1]:
import csv
import json
import re
from pathlib import Path

import duckdb

## Save required columns to a CSV file

In [ ]:
# Open up the ndjson file created by the Newspaper Harvester
with Path("data", "20230722015049", "results.ndjson").open() as json_file:

    # Open a CSV file for writing the filtered results
    with Path("front_pages.csv").open("w") as csv_file:
        csv_writer = csv.writer(csv_file)
        # Write the CSV column headers
        csv_writer.writerow(
            [
                "article_id",
                "title",
                "newspaper_id",
                "date",
                "category",
                "word_count",
                "page_id",
            ]
        )
        # Process the ndjson file line by line to avoid memory problems
        for line in json_file:
            # print(line)
            data = json.loads(line)
            try:
                # Articles in supplements can also have a page number of 1
                # These can be excluded by looking for an 'S' in the `pageSequence` field
                if str(data["pageSequence"]).endswith("S"):
                    continue
                # Select the fields we want to add tto the CSV
                row = [
                    data["id"],
                    data.get("heading", ""),
                    data["title"]["id"],
                    data["date"],
                    data["category"],
                    data["wordCount"],
                    re.search(r"\/page\/(\d+)$", data["trovePageUrl"]).group(1),
                ]
            except (TypeError, KeyError):
                # These are likely to be articles that are 'coming soon' with page links. Uncomment the next line to check
                # print(data)
                pass
            else:
                # Write the row of data
                csv_writer.writerow(row)

## Use DuckDB to create parquet files

The CSV file created by the step above is about 1.3gb. That's too big to easily share or use, so we'll convert it to a parquet file which will be much more compact. One way of doing this is to use a `COPY` statement with DuckDB.

In [ ]:
# This just loads all the data from the CSV into DuckDB then saves it as a parquet file.
duckdb.sql(
    "COPY (SELECT * FROM read_csv('front_pages.csv', AUTO_DETECT=TRUE)) TO 'front_pages.parquet' (FORMAT 'PARQUET', CODEC 'ZSTD');"
)

What I'm really interested in is how the number of words in each of Trove's article categories varies over time – in particular how do the 'Advertising` and 'Article categories compare? To make this easier to explore, we'll use DuckDB to run an SQL query over the CSV data, grouping the results by page, newspaper, and category, and then adding together the word counts. What we'll end up with is a breakdown for each page, giving the total number of words in each category.

In [ ]:
duckdb.sql(
    "COPY (SELECT date, page_id, newspaper_id, category, sum(word_count) AS total FROM read_csv('front_pages.csv', AUTO_DETECT=TRUE) GROUP BY date, page_id, newspaper_id, category) TO 'front_pages_totals.parquet' (FORMAT 'PARQUET', CODEC 'ZSTD');"
)